# Prática Independente - Regressão IV.

## Dados abertos do Airbnb no Rio de Janeiro:

#### Desde 2008, os hóspedes e anfitriões têm usado o Airbnb para expandir as possibilidades de viagem e apresentar uma forma mais única e personalizada de experimentar o mundo. Este conjunto de dados descreve a atividade de listagem e as métricas em Rio de Janeiro, RJ para 2020.

### Conteúdo:

#### O arquivo `'AB_RJ_2020.csv'` inclui todas as informações necessárias para descobrir mais sobre hosts, disponibilidade geográfica, métricas necessárias para fazer previsões e tirar conclusões.

#### Este conjunto de dados [públicos](http://insideairbnb.com/) faz parte do [Airbnb](https://www.airbnb.com.br/).

<img src="RegMetroRJ.png" width="1532" height="1037" align="center"/>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Tamanho padrões das figuras e a fonte de seus textos neste notebook
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#Funções que serão utilizadas ao longo do notebook

In [2]:
def EDA (df):
    """Função que retona algumas métricas de análise estatística descritiva personalizadas"""
    eda_df = {}
    eda_df['Amount_NaN'] = df.isnull().sum()
    eda_df['%_NaN'] = df.isnull().mean().round(2)
    eda_df['DType'] = df.dtypes
    eda_df['Amount_Data'] = df.count()
    
    # Outro ponto para ser verificado, porque para criar a coluna com a quantidade de valores unicos por coluna
    # Não utilizei a função df.unique() 
    colunas = sorted(df.columns.tolist(), key=str.lower, reverse=False)
    eda_df['Amount_Unique'] = filtered_result = list(map(lambda x: len(df[x].unique().tolist()), colunas))
    
    eda_df['Mean'] = np.round(df.mean(), 2)
    eda_df['Median'] = np.round(df.median(), 2)
    
    eda_df['Max'] = df.max()
    eda_df['Min'] = df.min()
    eda_df['STD'] = np.round(df.std(), 2)
    
    return pd.DataFrame(eda_df)

In [3]:
from scipy import stats

def normal(df, col, threshold = 0.05):

  """Função que retorna se uma distribuição é ou não normal"""
  try:
    zscore, p_value = stats.normaltest(df[col])
    #print('p_value:', p_value)
    #print("stats.normaltest(df['{}']):".format(col), stats.normaltest(df[col]))
        
    if p_value < threshold:
      result = 'A distribuição não é normal'
    else:
      result = 'A distribuição é normal' 
  except:
    zscore = p_value = np.nan
    result = 'Não aplicável'
  return result

In [4]:
#Instanciado objeto OrdinalEncoder
encoder = OrdinalEncoder()

def encode(data):
    """função que codifica valores não nulos e substitui no dataset original"""
    #retira os valores nulos dataset
    nao_nulos = np.array(data.dropna())
    #reshape dos dados para codificação
    impute_reshape = nao_nulos.reshape(-1,1)
    #codifica os dados
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Atribui os valores codificados de volta a valores não nulos
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#### Exercício 1 - Realize uma análise exploratória nos dados abertos do AirBNB para a cidade do Rio de Janeiro.

In [5]:
dados = pd.read_csv('AB_RJ_2020.csv', index_col= 'id')
dados.head()

,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,,,,
17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",68997,Matthias,"Rio de Janeiro, Brazil",-22.96592,-43.17896,Entire home/apt,$500.00,5,259,2020-12-26,2.03,1,0
25026,Beautiful Modern Decorated Studio in Copa,3746246,Ghizlane,"Rio de Janeiro, Brazil",-22.97712,-43.19045,Entire home/apt,$160.00,7,238,2020-02-15,1.85,11,335
35636,Cosy flat close to Ipanema beach,153232,Patricia,"Rio de Janeiro, Brazil",-22.98816,-43.19359,Entire home/apt,$259.00,2,181,2020-03-15,2.07,1,267
35764,COPACABANA SEA BREEZE - RIO - 20 X Superhost,153691,Patricia Miranda & Paulo,"Rio de Janeiro, Brazil",-22.98127,-43.19046,Entire home/apt,$157.00,3,346,2020-12-20,2.78,1,89
41198,"Modern 2bed,Top end of Copacabana",178975,Nicky,NaN,-22.97962,-43.19230,Entire home/apt,"$1,035.00",3,18,2016-02-09,0.19,2,365


In [6]:
print(f'Instâncias: {dados.shape[0]}')
print(f'Variáveis: {dados.shape[1]}')

Instâncias: 25784
Variáveis: 14


In [7]:
EDA(dados)

,Amount_NaN,%_NaN,DType,Amount_Data,Amount_Unique,Mean,Median,Max,Min,STD
availability_365,0,0.00,int64,25784,366,215.30,247.00,365,0,1.408900e+02
calculated_host_listings_count,0,0.00,int64,25784,48,6.58,1.00,200,1,2.000000e+01
host_id,0,0.00,int64,25784,16946,96308131.49,58502919.50,381289871,3607,1.041834e+08
host_name,8,0.00,object,25776,5076,NaN,NaN,NaN,NaN,NaN
last_review,9932,0.39,object,15852,1447,NaN,NaN,NaN,NaN,NaN
latitude,0,0.00,float64,25784,9782,-22.97,-22.97,-22.7498,-23.0729,3.000000e-02
longitude,0,0.00,float64,25784,12093,-43.25,-43.20,-43.1049,-43.7048,1.000000e-01
minimum_nights,0,0.00,int64,25784,67,4.77,2.00,1000,1,1.904000e+01
name,29,0.00,object,25755,25086,NaN,NaN,NaN,NaN,NaN
neighbourhood,11675,0.45,object,14109,270,NaN,NaN,NaN,NaN,NaN


1) Temos 25.784 instâncias no dataset e 14 variáveis a priori;

2) Temos muitos dados nulos nas variáveis last_review, neighbourhood e reviews_per_month

3) Precisamos tratar os dados da coluna price para tranformá-los em tipo numérico para utilizarmos no modelo de regressão com o tipo correto

4) Temos uma media de 215 em que os hosts estão disponíveis, por ano, com uma mediana de 247 e moda de 365. A maioria está disponivel o ano todo, porém possuímos outliers com o número de 0 (seria um erro nos dados?) disponíveis por ano que podem estar puxando a media pra baixo.

5) Olhando mediana e media podemos indicar que a maioria dos dados de minímo de noite (para a paga do valor indicado) está entre 2 e 5).

In [8]:
dados['host_name'].value_counts()

Daniel           318
Maria            270
Marcelo          243
Ricardo          212
Carlos           196
                ... 
Hakumi             1
Juliana De         1
Rubs               1
Kauany             1
Fabiana Karla      1
Name: host_name, Length: 5075, dtype: int64

Ao contar o número de hosts por pessoas vemos muitas que se repetem. Isso pode se dar porque só o primeiro nome está contido e assim são pessoas diferentes com nomes iguais ou pode haver pessoas com maior número de locais disponiveis na plataforma Airbnd e outras que colocam a vaga de forma mais eventual.

#### Exercício 2 -Faça a limpeza dos dados.

1) Ao explorar a coluna neighbourhood encontramos hosts do Bairro Rio de Janeiro escrito de duas formas diferentes: 'río (ascento agudo no i) de janeiro e rio (sem acento agudo no i) de janeiro. Vamos padronizar os dados retirando o acento agudo desses i's. 

2) Vamos retirar colunas que não serão utilizadas no modelo e passar os nomes daquelas que possuem variáveis categóricas para variáveis numéricas.

3) Além disso iremos aproveitar e passar todos os seus nomes para minúsculos. Assim evitamos que os mesmos tipos de quartos ou mesmos bairros sejam codificados como diferentes por estarem escritos de forma diferente.

In [9]:
#Retirando colunas que não serão utilizadas
dados.drop(labels= ['host_name', 'host_id', 'name', 'last_review'], axis= 1, inplace= True)

#Passando valores de colunas categóricas para letras minúsculas
colunas_minusculas = ['neighbourhood', 'room_type']
for coluna in colunas_minusculas:
  dados[coluna].str.lower()

#Padronizando nome do bairro rio de janeiro
dados['neighbourhood']= dados['neighbourhood'].str.replace('í', 'i', regex= True)

Vamos transformar a coluna price de object para numérica

In [10]:
#Modificando coluna de preço de object para int
dados['price']= dados['price'].str.replace('$', '', regex= True)
dados['price']= dados['price'].str.replace(',', '', regex= True)
dados['price'] = pd.to_numeric(dados['price'])

dados['price'].head()

id
17878     500.0
25026     160.0
35636     259.0
35764     157.0
41198    1035.0
Name: price, dtype: float64

Um dos pressupostos da regressão linear é que a distribuição das variáveis sejam normalizadas: vamos fazer o teste estatístico pra sabermos se a distribuição de nossa variável resposta (o preço dos hosts) é ou não normal.

In [11]:
normal(dados, 'price')

'A distribuição não é normal'

Como não temos uma distribuição normalizada, provavelmente teremos problemas com o modelo no exercício 4. Para comparar a performance do modelo, vamos fazer um com as medidas originais dos preços (distribuição não normal) e depois calcularmos o logaritmo de cada preço e refazer o modelo baseado nessa medida e vermos se o modelo tem melhoria na performance. Mas primeiro vamos continuar o processo de limpeza e tratamento dos dados.

Vamos preencher os valores faltantes numericos usando o algoritmo [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) que distribui os dados em pontos n-dimensionais (sendo n o total de variáveis independentes) e prediz um valor para o dado faltante baseado na sua proximidade nesse espaço com pontos que não possuem dados faltantes. Primeiro, porém, temos de codificar as variáveis categóricas com dados numéricos em nosso dataset.

In [12]:
#Codificando colunas categóricas de nossos dados
colunas_cat = ['neighbourhood', 'room_type']

for coluna in colunas_cat:
  encode(dados[coluna])

dados.sample(4)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,
2685031,214,-22.94082,-43.17752,2.0,600.0,2,0,NaN,1,0
2395690,214,-22.96957,-43.18522,2.0,513.0,5,0,NaN,1,363
2211732,181,-22.98454,-43.19597,0.0,1590.0,2,80,1.80,1,248
33689287,NaN,-23.01977,-43.46170,2.0,65.0,2,23,1.13,2,287


Agora que temos nossa coluna de bairros com valores numéricos representando cada bairro, podemos fazer o preenchimento dos valores faltantes utilizando o KNNImputer

In [13]:
#Instanciando objeto KNNImputer com 7 números vizinhos de parametro, pesos iguais para todos os vizinhos dentro desses 7
#E distância euclidiana para calcular as distâncias entre os pontos
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

#Treinando modelo
imputer.fit(dados)

#Transformando o dataset original com os valores nulos implementados
dados_transf = imputer.transform(dados)

#Retornando os dados para um dataframe pandas com as categorias em formato numérico
dados_transf = pd.DataFrame(imputer.fit_transform(dados_transf), columns = dados.columns)
dados_transf.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,181.0,-22.96592,-43.17896,0.0,500.0,5.0,259.0,2.03,1.0,0.0
1,181.0,-22.97712,-43.19045,0.0,160.0,7.0,238.0,1.85,11.0,335.0
2,181.0,-22.98816,-43.19359,0.0,259.0,2.0,181.0,2.07,1.0,267.0
3,181.0,-22.98127,-43.19046,0.0,157.0,3.0,346.0,2.78,1.0,89.0
4,168.6,-22.97962,-43.19230,0.0,1035.0,3.0,18.0,0.19,2.0,365.0


Agora vamos resumir a coluna de neighbourhood com a proporção que aquele bairro aparece no dataset

In [14]:
frequencia_bairros = pd.DataFrame()
frequencia_bairros['neighbourhood'] = dados_transf['neighbourhood'].value_counts(normalize = True).index.tolist()
frequencia_bairros['frequencia_neighbourhood'] = dados_transf['neighbourhood'].value_counts(normalize = True).tolist()
frequencia_bairros.head()

,neighbourhood,frequencia_neighbourhood
0,181.0,0.226575
1,54.0,0.087845
2,15.0,0.029165
3,214.0,0.028971
4,94.0,0.021641


Junção dos dataframes dados_transf e frequencia_bairros com o método .merge() que mescla DataFrames ou objetos nomeados do tipo Series com uma junção de estilo de banco de dados. Depois podemos eliminar a coluna neighbourhood.

In [15]:
dados_transf = dados_transf.merge(frequencia_bairros, 
                                on = 'neighbourhood'
                               )
dados_transf.drop(labels= 'neighbourhood', axis= 1, inplace= True)

dados_transf.head()

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,frequencia_neighbourhood
0,-22.96592,-43.17896,0.0,500.0,5.0,259.0,2.03,1.0,0.0,0.226575
1,-22.97712,-43.19045,0.0,160.0,7.0,238.0,1.85,11.0,335.0,0.226575
2,-22.98816,-43.19359,0.0,259.0,2.0,181.0,2.07,1.0,267.0,0.226575
3,-22.98127,-43.19046,0.0,157.0,3.0,346.0,2.78,1.0,89.0,0.226575
4,-22.98599,-43.20299,0.0,6685.0,2.0,80.0,0.72,5.0,0.0,0.226575


In [16]:
EDA(dados_transf)

,Amount_NaN,%_NaN,DType,Amount_Data,Amount_Unique,Mean,Median,Max,Min,STD
latitude,0,0.0,float64,25784,366,-22.97,-22.97,-22.749820,-23.072860,0.03
longitude,0,0.0,float64,25784,48,-43.25,-43.20,-43.104860,-43.704790,0.10
room_type,0,0.0,float64,25784,93,0.58,0.00,3.000000,0.000000,0.94
price,0,0.0,float64,25784,9782,879.82,380.00,593266.000000,0.000000,5179.76
minimum_nights,0,0.0,float64,25784,12093,4.77,2.00,1000.000000,1.000000,19.04
number_of_reviews,0,0.0,float64,25784,67,11.96,1.00,431.000000,0.000000,29.37
reviews_per_month,0,0.0,float64,25784,273,0.44,0.18,14.550000,0.010000,0.68
calculated_host_listings_count,0,0.0,float64,25784,2187,6.58,1.00,200.000000,1.000000,20.00
availability_365,0,0.0,float64,25784,1297,215.30,247.00,365.000000,0.000000,140.89
frequencia_neighbourhood,0,0.0,float64,25784,4,0.06,0.01,0.226575,0.000039,0.09


#### Exercício 3 - Analise os dados e realize a anonimização dos mesmos. 

Já o fizemos ao retirar as colunas que dizem respeito ao nomes dos hosts e nome dos donos dos hosts no exercício 2.

#### Exercício 4 - Crie um modelo de regressão para os preços dos apartamentos.

####Predição preços unidade original


1) Separar um dataframe com as variáveis independentes e ou array numpy com a variável resposta (preço dos hosts)

2) Separar os dados em subconjuntos de treino e de teste

3) Padronizar os dados das variáveis independentes com o z_score: Vamos normalizar os dados para ter nossas variáveis preditoras na mesma unidade, neste caso, seus desvios em relação a media da própria distribuição: o [z-score](https://www.investopedia.com/terms/z/zscore.asp#:~:text=A%20Z%2Dscore%20is%20a,standard%20deviations%20from%20the%20mean.&text=A%20Z%2Dscore%20of%201.0,standard%20deviation%20from%20the%20mean.)

4) Treinar modelo

5) Analisar seu R² e seus coeficientes

6) Fazer predições e analisar erro medio absoluto (quantos reais o modelo está errando para cima ou para baixo em media)

In [17]:
#Para que haja os mesmos resultados quando o notebook for rodado novamente
np.random.RandomState(3)

#Variáveis que utilizaremos para a previsão
var_independentes = dados_transf.columns.to_list()

#Retirando variável resposta
var_independentes.remove('price')

#Dataframe com as variáveis que utilizaremos para revisão
X= dados_transf[var_independentes]

#Variável que queremos prever
y= dados_transf['price']

#Separação de dados de treino (70%) e de teste (30%)
X_treino, X_teste, y_treino, y_teste = train_test_split(X.values, y, test_size = 0.3)

#Instanciando objeto StandardScaler
scaler = StandardScaler().fit(X_treino)

#Z-Scores dos dados de treino e de teste
x_treino_norm = scaler.transform(X_treino)
x_teste_norm = scaler.transform(X_teste)

#Instanciando objeto Linear Regression e ajustando modelo com os dados de treino
reg = LinearRegression().fit(x_treino_norm, y_treino)

#R² do nosso modelo
print(f'R²: {reg.score(x_treino_norm, y_treino)}')

#Coeficientes de acordo com a variável
importancia_df = pd.DataFrame()
importancia_df['colunas'] = var_independentes
importancia_df['importancia'] = list(reg.coef_)
print(importancia_df)

R²: 0.004566661900818114
                          colunas  importancia
0                        latitude  -144.683928
1                       longitude    24.881926
2                       room_type  -119.148665
3                  minimum_nights     4.793618
4               number_of_reviews  -164.451000
5               reviews_per_month    25.548173
6  calculated_host_listings_count   224.829157
7                availability_365   158.400614
8        frequencia_neighbourhood   -19.043905


Cruzando coeficientes das variáveis em relação à variável resposta: como cada variável independente influencia a variável dependente:

1) R² -> Vemos que nosso R² (medida que nos indica o quantos nossas variáveis independentes estão explicando a variação de nossa variável resposta) tem um valor bem baixo (o ideal seria mais próximo de 1 possível). Porém, como colocamos nossas variáveis numa escala diferente isto acaba afetando a métrica do R².

2) Coeficientes:

  a) Conforme aumenta a latitude o preço tem a tendência de diminuir R$144 reais e com o aumento da longitude o preço tende a diminuir em R$24 reais. Isso nos indica, claro, padrões de comportamento dos preços de acordo com a região que o host está presente.

  b) (0 - 'Entire home/apt'), (1 - 'Private room', (2 - 'Shared room'), (3 - 'Hotel room'). A codificação dos tipos de quartos segue um ordem qualitativa de quanto maior o número que o quarto foi codificado, menos privado e menor ele o é. Nesse sentido, o coeficiente de room_type nos indica que quanto menor for o quarto e menos privado tal fator tem a tendência de diminuir o preço em R$119 reais.

  c) O coeficiente de minímo por noite nos indica que quanto o maior o número de minímo per noite a tendência é a subida do preço em R$4 reais. Portanto, dentre os dados e os bairros analisados, a tendência de aumento no preço se ficarmos uma noite a mais no host é de 4 reais.

  d) A tendência conforme o aumento do número de reviews sobe é de diminuição do preço em aproximadamente R$164 reais. Duas hipóteses podem ser levantadas aqui: d1) os hosts mais baratos são aqueles com maior frequência de clientes e esses clientes dão o retorno positivo; ou d2) os hosts mais baratos possuem maior número de clientes, mas estes retornam com reviews negativos. Para confirmar se os reviews são positivos ou negativos precisaríamos olhar sua distribuição (se forem reviews em formatos de notas) e/ou aplicar um modelo de análise de sentimento (se os reviews forem comentários).

  e) A tendência dos preços é de aumentar R$158 reais quanto o host permanece mais dias disponiveis por ano.

  f) Conforme o aumento na frequência de hosts em um bairro, menor o seu preço. Isto nos indica a questão da concorrência na região: quanto maior o número de hosts mais próximos o preço geralmente diminui devido à disputa com a concorrência. Este comportamento é comum em vários tipos de negócios. A tendência é o preço diminuir seu valor em R$19 reais se houver maior número de concorrentes. Uma variação não muito alta.

Vamos fazer as predições e então ver o erro medio absoluto (pegando o valor exatamente em reais que estamos errando em media dos preços) do nosso modelo.

In [18]:
np.random.RandomState(5)
#Predições
y_pred = reg.predict(x_teste_norm)

#Erro medio absoluto
erro_abs_medio = mean_absolute_error(y_teste, y_pred)
print(f'Erro absoluto medio: {erro_abs_medio}')

Erro absoluto medio: 800.7232311953723


Estamos errando os preços dos hosts em media (tanto pra positivo quanto negativo já que pegamos o valor absoluto das diferenças dos valores reais e das predições) R$800.72. É um erro de fato muito alto quanto o nosso R² baixíssimo estava nos indicando. Vamos tentar (re)escalonar os preços colocando eles em seus valores logarítmicos e medir novamente a performance do modelo.

####Predição com os preços em Log

Como indicado quando analisamos a dstribuição dos preços na sua unidade original, eles não apresentam uma distribuição normal. Agora iremos fazer os mesmos passos feitos acima, porém com os preços reescalonados para seu valor logarítmico e ver se a performance do modelo é melhorada.

In [23]:
dados_transf['price_log'] = np.log(dados_transf['price'])
dados_transf['price_log'].head()

0    6.214608
1    5.075174
2    5.556828
3    5.056246
4    8.807621
Name: price_log, dtype: float64

In [26]:
#Retirando dados com valores infinitos dos dados com os preços em logaritmicos para utilizar no modelo de regressão linear
dados_transf.replace([np.inf, -np.inf], np.nan, inplace= True)
dados_transf.dropna(subset=['price_log'], axis= 0, inplace= True)

#Conferindo se perdemos muitas linhas ao fazer essa remoção
len(dados_transf)

25777

In [27]:
np.random.RandomState(6)
#Dataframe com as variáveis que utilizaremos para revisão
X= dados_transf[var_independentes]

#Variável que queremos prever
y= dados_transf['price_log']

#Separação de dados de treino (70%) e de teste (30%)
X_treino, X_teste, y_treino, y_teste = train_test_split(X.values, y, test_size = 0.3, random_state= 2)

#Padronizando escala de variáveis indepedentes com z_score
scaler = StandardScaler().fit(X_treino)

#Z-Scores dos dados de treino e de teste das independentes
x_treino_norm = scaler.transform(X_treino)
x_teste_norm = scaler.transform(X_teste)

#Instanciando objeto Linear Regression e ajustando modelo com os dados de treino
reg = LinearRegression().fit(x_treino_norm, y_treino)
np.random.RandomState(2)

#R² do nosso modelo
print(f'R²: {reg.score(x_treino_norm, y_treino)}')

#Predições do modelo
y_pred = reg.predict(x_teste_norm)

#Erro medio absoluto do "novo" modelo
erro_abs_medio = mean_absolute_error(y_teste, y_pred)
print(f'Erro absoluto medio: {erro_abs_medio}')

R²: 0.2884205898466399
Erro absoluto medio: 0.669144653093172


O nosso R² já demonstra um aumento considerável com a alteração feita, mas ainda temos escalas diferentes. Então, para termos a real dimensão de melhoria ou não no modelo, vamos analisar o erro medio absoluto:

Temos um modelo que performa muito melhor que aquele com os preços em sua unidade original. Porém, como o erro aqui é bem pequeno, o nosso modelo pode estar tendo aquilo que é chamado de overffiting: ele está se ajustando tão bem aos dados que passamos que não conseguiria calcular com a mesma acurácia dados novos com comportamentos diferentes.